#Code Cafe

In [ ]:
!wget http://arunponnusamy.com/files/mmod_human_face_detector.dat

--2022-03-26 19:11:36--  http://arunponnusamy.com/files/mmod_human_face_detector.dat
Resolving arunponnusamy.com (arunponnusamy.com)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to arunponnusamy.com (arunponnusamy.com)|185.199.109.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.arunponnusamy.com/files/mmod_human_face_detector.dat [following]
--2022-03-26 19:11:36--  https://www.arunponnusamy.com/files/mmod_human_face_detector.dat
Resolving www.arunponnusamy.com (www.arunponnusamy.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to www.arunponnusamy.com (www.arunponnusamy.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 729940 (713K) [application/octet-stream]
Saving to: ‘mmod_human_face_detector.dat’

mmod_human_face_det 100%[===================>] 712.83K  --.-KB/s    in 0.04s   

2022-03-26 19:11:37 (15.5 MB/s) - ‘mmod_human_face_dete

##Image Processing
https://learnopencv.com/facial-landmark-detection/



In [ ]:
!pip install imutils
!pip install dlib
!pip install opencv-python
!pip install imageio
!pip install opencv-contrib-python

In [ ]:
from imutils import face_utils                                                                                                                                                               
import dlib
import cv2
import imageio
from google.colab.patches import cv2_imshow

model = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(model)

image_jon = imageio.imread("jon.jpg")
image_salir = imageio.imread("salir.jpg")

factor = 0.25
image_jon = cv2.resize(image_jon, (int(factor*image_jon.shape[1]), int(factor*image_jon.shape[0])))
image_salir = cv2.resize(image_salir, (int(factor*image_salir.shape[1]), int(factor*image_salir.shape[0])))

print(f"image_jon: {image_jon.shape}")
print(f"image_salir: {image_salir.shape}")

imageio.imwrite("jon_scaled.jpg", image_jon)
imageio.imwrite("salir_scaled.jpg", image_salir)

# image = cv2.resize(image.shape[])

images = (image_jon, image_salir)
for i,name in enumerate(['jon', 'salir']):
  image_gray = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
  image_rgb = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)

  rects = detector(image_gray, 0)
  image_gray = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
  for (i, rect) in enumerate(rects):
      # Make the prediction and transfom it to numpy array
      shape = predictor(image_gray, rect)
      shape = face_utils.shape_to_np(shape)


      # Draw on our image, all the finded cordinate points (x,y)
      printStr = ""
      for (x, y) in shape:
          cv2.circle(image_rgb, (x, y), 2, (0, 255, 0), -1)
          printStr += f"  {x}  {y}\n"
      xmax = image_rgb.shape[0]-1
      ymax = image_rgb.shape[1]-1
      printStr += f"  {ymax}  {xmax}\n"
      printStr += f"  {0}  {xmax}\n"
      printStr += f"  {ymax}  {0}\n"
      printStr += f"  {0}  {0}\n"
      
      with open(f"{name}_pts.txt", "w") as file:
          file.writelines(printStr)

  cv2_imshow(image_rgb)








In [ ]:
from os import fpathconf
import random
import numpy as np

def rect_contains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True
# Draw a point
def draw_point(img, p, color ) :
    cv2.circle( img, p, 2, color, cv2.FILLED, cv2.LINE_AA, 0 )
# Draw delaunay triangles
def xrange(x):
    return iter(range(x))

def help(pts,file):
  d = pts_to_idx
  buf = " ".join([str(d[(int(x[0]),int(x[1]))]) for x in pts]) + "\n"
  file.write(buf)

def draw_delaunay(img, subdiv, delaunay_color , fname) :
    fp = open(fname, 'w')
    triangleList = subdiv.getTriangleList();
    size = img.shape
    r = (0, 0, size[1], size[0])
    for t in triangleList :
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        # print(f"{pts_to_idx[(int(pt1[0]),int(pt1[1]))]}")
        help([pt1, pt2, pt3], fp)
        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3) :
            cv2.line(img, pt1, pt2, delaunay_color, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt2, pt3, delaunay_color, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt3, pt1, delaunay_color, 1, cv2.LINE_AA, 0)
    fp.close()
# Draw voronoi diagram
def draw_voronoi(img, subdiv) :
    ( facets, centers) = subdiv.getVoronoiFacetList([])
    for i in xrange(0,len(facets)) :
        ifacet_arr = []
        for f in facets[i] :
            ifacet_arr.append(f)
        ifacet = np.array(ifacet_arr, np.int)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        cv2.fillConvexPoly(img, ifacet, color, cv2.LINE_AA, 0);
        ifacets = np.array([ifacet])
        cv2.polylines(img, ifacets, True, (0, 0, 0), 1, cv2.LINE_AA, 0)
        cv2.circle(img, (centers[i][0], centers[i][1]), 3, (0, 0, 0), cv2.FILLED, cv2.LINE_AA, 0)


In [ ]:

for name in ['jon', 'salir']:
  delaunay_color = (255,255,255)
  points_color = (0, 0, 255)
  img = cv2.imread(f"{name}_scaled.jpg")
  img_orig = img.copy()
  size = img.shape
  rect = (0, 0, size[1], size[0])
  # Create an instance of Subdiv2D
  subdiv = cv2.Subdiv2D(rect);
  # Create an array of points.
  points = [];

  pts_to_idx = {}
  with open(f"{name}_pts.txt") as file :
      for i,line in enumerate(file) :
          x, y = line.split()
          pts_to_idx[(int(x),int(y))] = i
          points.append((int(x), int(y)))

  for p in points :
      subdiv.insert(p)
      # Show animation
      if False : #animation?
          img_copy = img_orig.copy()
          # Draw delaunay triangles
          draw_delaunay( img_copy, subdiv, (255, 255, 255) );
          cv2.cv2_imshow(img_copy)
          cv2.waitKey(100)

  draw_delaunay( img, subdiv, (255, 255, 255), f'tri_{name}.txt');
  # Draw points
  for p in points :
      draw_point(img, p, (0,0,255))

  cv2_imshow(img)




In [ ]:
import sys

# Read points from text file
def readPoints(path) :
    # Create an array of points.
    points = [];
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))

    return points

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst


# Warps and alpha blends triangular regions from img1 and img2 to img
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))


    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []


    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)

    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2

    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask

In [ ]:
f1 = 'jon_scaled.jpg'
f2 =  'salir_scaled.jpg'

numFrames = 125


for frame in range(numFrames):
  alpha = frame / numFrames

  img1 = cv2.imread(f1);
  img2 = cv2.imread(f2);


  img1 = np.float32(img1)
  img2 = np.float32(img2)

  points1 = readPoints('jon_pts.txt')
  points2 = readPoints('salir_pts.txt')
  points = [];

  for i in range(0, len(points1)):
      x = ( 1 - alpha ) * points1[i][0] + alpha * points2[i][0]
      y = ( 1 - alpha ) * points1[i][1] + alpha * points2[i][1]
      points.append((x,y))

  imgMorph = np.zeros(img1.shape, dtype = img1.dtype)


  with open("tri_jon.txt") as file :
      for line in file :
          x,y,z = line.split()
          
          x = int(x)
          y = int(y)
          z = int(z)
          
          t1 = [points1[x], points1[y], points1[z]]
          t2 = [points2[x], points2[y], points2[z]]
          t = [ points[x], points[y], points[z] ]

          # Morph one triangle at a time.
          morphTriangle(img1, img2, imgMorph, t1, t2, t, alpha)

  # cv2_imshow(np.uint8(imgMorph))
  cv2.imwrite(f"frames_small/frame_{int(alpha*numFrames):03d}.png", imgMorph)
  print(f"finished frame {int(alpha*numFrames):03d}.png")



finished frame 000.png
finished frame 001.png
finished frame 002.png
finished frame 003.png
finished frame 004.png
finished frame 005.png
finished frame 006.png
finished frame 007.png
finished frame 008.png
finished frame 009.png
finished frame 010.png
finished frame 011.png
finished frame 012.png
finished frame 013.png
finished frame 014.png
finished frame 015.png
finished frame 016.png
finished frame 017.png
finished frame 018.png
finished frame 019.png
finished frame 020.png
finished frame 021.png
finished frame 022.png
finished frame 023.png
finished frame 024.png
finished frame 025.png
finished frame 026.png
finished frame 027.png
finished frame 028.png
finished frame 029.png
finished frame 030.png
finished frame 031.png
finished frame 032.png
finished frame 033.png
finished frame 034.png
finished frame 035.png
finished frame 036.png
finished frame 037.png
finished frame 038.png
finished frame 039.png
finished frame 040.png
finished frame 041.png
finished frame 042.png
finished fr

In [ ]:
!zip -r frames_small.zip ./frames_small

  adding: frames_small/ (stored 0%)
  adding: frames_small/frame_122.png (deflated 4%)
  adding: frames_small/frame_053.png (deflated 3%)
  adding: frames_small/frame_102.png (deflated 3%)
  adding: frames_small/frame_039.png (deflated 3%)
  adding: frames_small/frame_082.png (deflated 3%)
  adding: frames_small/frame_025.png (deflated 3%)
  adding: frames_small/frame_114.png (deflated 3%)
  adding: frames_small/frame_024.png (deflated 3%)
  adding: frames_small/frame_073.png (deflated 3%)
  adding: frames_small/frame_076.png (deflated 3%)
  adding: frames_small/frame_003.png (deflated 4%)
  adding: frames_small/frame_117.png (deflated 3%)
  adding: frames_small/frame_031.png (deflated 3%)
  adding: frames_small/frame_116.png (deflated 3%)
  adding: frames_small/frame_001.png (deflated 5%)
  adding: frames_small/frame_051.png (deflated 3%)
  adding: frames_small/frame_050.png (deflated 3%)
  adding: frames_small/frame_060.png (deflated 3%)
  adding: frames_small/frame_089.png (deflated

In [ ]:
from google.colab import files
files.download("frames_small.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>